# Show Raw Data

In [11]:
import sys
import math
from pathlib import Path

from eztrack import (
    preprocess_raw,
    lds_raw_fragility,
    write_result_fragility,
    plot_result_heatmap,
)
from eztrack.utils import warn
from mne_bids import read_raw_bids, BIDSPath, get_entity_vals

sys.path.append('../')

from analysis.fragility.run_fragility_analysis import run_analysis

In [2]:
import matplotlib
import mne
mne.viz.set_3d_backend("notebook")

In [3]:
# %gui qt
%matplotlib qt

In [12]:
def preprocess_data(bids_path, reference, deriv_path=None, figures_path=None, verbose=True, overwrite=False):
    subject = bids_path.subject

    # set where to save the data output to
    if deriv_path is None:
        deriv_path = (
                bids_path.root
                / "derivatives"
                / "1000Hz"
                / "fragility"
                / reference
                / f"sub-{subject}"
        )
    if figures_path is None:
        figures_path = (
                bids_path.root
                / "derivatives"
                / "figures"
                / "1000Hz"
                / "fragility"
                / reference
                / f"sub-{subject}"
        )

    # use the same basename to save the data
    deriv_basename = bids_path.basename

    # if len(list(deriv_path.rglob(f'{deriv_basename}*'))) > 0 and not overwrite:
    #     warn('Need to set overwrite to True if you want '
    #          f'to overwrite {deriv_basename}')
    #     return

    # load in the data
    raw = read_raw_bids(bids_path)
    print(raw.ch_names)
    raw = raw.pick_types(seeg=True, ecog=True, eeg=True, misc=False, exclude=[])
    print(raw.ch_names)
    raw.load_data()
    print(raw.ch_names)

    # pre-process the data using preprocess pipeline
    datatype = bids_path.datatype
    print('Power Line frequency is : ', raw.info["line_freq"])
    raw = preprocess_raw(raw, datatype=datatype,
                         verbose=verbose, method="simple", drop_chs=False)
    print(raw.ch_names)
#     raw = raw.pick_types(seeg=True, ecog=True, eeg=True, misc=True, exclude=[])
    return raw

In [13]:
# the root of the BIDS dataset
root = Path("/Users/adam2392/OneDrive - Johns Hopkins/sickkids/")

# define BIDS entities
subjects = [
    # "E1",
    # 'E2',
    # 'E3',
    'E4',
    # 'E5', 'E6'
]

session = "postsurgery"  # only one session

# pre, Sz, Extraoperative, post
# task = "pre"
acquisition = "ecog"
datatype = "ieeg"
extension = ".vhdr"

# analysis parameters
reference = 'average'

sessions = [
    'presurgery',
            'extraoperative',
            'intraoperative',
            'postsurgery']

# get the runs for this subject
subjects = get_entity_vals(root, "subject")
for subject in subjects:
    if subject != 'E4':
        continue
    ignore_subs = [sub for sub in subjects if sub != subject]
    ignore_sessions = [ses for ses in sessions if ses != session]
    ignore_set = {
        'ignore_subjects': ignore_subs,
        'ignore_sessions': ignore_sessions,
    }
    all_tasks = get_entity_vals(root, "task", **ignore_set)
    tasks = all_tasks
    # tasks = ['pre']

    for task in tasks:
        print(f"Analyzing {task} task.")
        ignore_tasks = [tsk for tsk in all_tasks if tsk != task]
        ignore_set['ignore_tasks'] = ignore_tasks
        runs = get_entity_vals(
            root, 'run', **ignore_set
        )
        print(ignore_subs)
        print(ignore_tasks)
        print(f'Found {runs} runs for {task} task.')

        for idx, run in enumerate(runs):
            # create path for the dataset
            bids_path = BIDSPath(
                subject=subject,
                session=session,
                task=task,
                run=run,
                datatype=datatype,
                acquisition=acquisition,
                suffix=datatype,
                root=root,
                extension=extension,
            )
            print(f"Analyzing {bids_path}")

            raw = preprocess_data(bids_path, reference=reference)
            
            break
        break
    break

Analyzing post task.
['E1']
[]
Found ['01'] runs for post task.
Analyzing /Users/adam2392/OneDrive - Johns Hopkins/sickkids/sub-E4/ses-postsurgery/ieeg/sub-E4_ses-postsurgery_task-post_acq-ecog_run-01_ieeg.vhdr
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'F2AL1', 'F2AL2', 'F2AL3', 'F2AL4', 'F2AL5', 'F2AL6', 'F2BC1', 'F2BC2', 'F2BC3', 'F2BC4', 'F2BC5', 'F2BC6', 'F2CL1', 'F2CL2', 'F2CL3', 'F2CL4', 'F2CL5', 'F2CL6', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'C101', 'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C10

<ipython-input-12-7f59e84365d7>:34: RuntimeWarning: Did not find any events.tsv associated with sub-E4_ses-postsurgery_task-post_acq-ecog_run-01.

The search_str was "/Users/adam2392/OneDrive - Johns Hopkins/sickkids/sub-E4/**/sub-E4_ses-postsurgery*events.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-12-7f59e84365d7>:34: RuntimeWarning: Did not find any electrodes.tsv associated with sub-E4_ses-postsurgery_task-post_acq-ecog_run-01.

The search_str was "/Users/adam2392/OneDrive - Johns Hopkins/sickkids/sub-E4/**/sub-E4_ses-postsurgery*electrodes.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-12-7f59e84365d7>:34: RuntimeWarning: Did not find any coordsystem.json associated with sub-E4_ses-postsurgery_task-post_acq-ecog_run-01.

The search_str was "/Users/adam2392/OneDrive - Johns Hopkins/sickkids/sub-E4/**/sub-E4_ses-postsurgery*coordsystem.json"
  raw = read_raw_bids(bids_path)


['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'F2AL1', 'F2AL2', 'F2AL3', 'F2AL4', 'F2AL5', 'F2AL6', 'F2BC1', 'F2BC2', 'F2BC3', 'F2BC4', 'F2BC5', 'F2BC6', 'F2CL1', 'F2CL2', 'F2CL3', 'F2CL4', 'F2CL5', 'F2CL6']
Power Line frequency is :  60
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'F2AL1', 'F2AL2', 'F2AL3', 'F2AL4', 'F2AL5', 'F2AL6', 'F2BC1', 'F2BC2', 'F2BC3', 'F2BC4', 'F2BC5', 'F2BC6', 'F2CL1', 'F2CL2', 'F2CL3', 'F2CL4', 'F2CL5',

In [14]:
print((raw.info['sfreq']))
print(raw.info['bads'])

2049.1803278688526
['C1', 'C2', 'C3', 'C4', 'C5', 'C9', 'C10', 'C11', 'C12', 'C13', 'C17', 'C18', 'C19', 'C20', 'C25', 'C26', 'C33', 'C34', 'C35', 'C36', 'C41', 'C42', 'C47', 'F2AL1', 'F2AL2', 'F2AL3', 'F2AL4', 'F2AL5', 'F2AL6', 'F2BC1', 'F2BC2', 'F2BC3', 'F2BC4', 'F2BC5', 'F2BC6', 'F2CL1', 'F2CL2', 'F2CL3', 'F2CL4', 'F2CL5', 'F2CL6']


In [15]:
deriv_root = root / 'derivatives' / "figures" \
                / f"{int(raw.info['sfreq'])}Hz" \
                / "raw" \
                / reference \
                / f"sub-{subject}"
deriv_root.mkdir(exist_ok=True, parents=True)
fig_basename = bids_path.copy().update(extension='.pdf').basename
print(deriv_root)
print(fig_basename)

/Users/adam2392/OneDrive - Johns Hopkins/sickkids/derivatives/figures/2049Hz/raw/average/sub-E4
sub-E4_ses-postsurgery_task-post_acq-ecog_run-01_ieeg.pdf


In [16]:
scale = 500e-6
fig = raw.plot(
    scalings={
        'ecog': scale, 
        'seeg': scale
    }, n_channels=len(raw.ch_names)
)
fig.savefig(deriv_root / fig_basename)

In [12]:
print(raw.info['bads'])

['C20', 'C27', 'C28']
